In [ ]:
import pandas as pd

from fastai.tabular.all import *

In [ ]:
DATA_PATH = '../data/preprocessed_training_data.pkl'

In [ ]:
df = pd.read_pickle(DATA_PATH)

In [ ]:
df.info()

In [ ]:
class_count_df = df.groupby('install').count()
class_count_df

n_0, n_1 = class_count_df.iloc[0, 0], class_count_df.iloc[1, 0]

w_0 = (n_0 + n_1) / (2.0 * n_0)
w_1 = (n_0 + n_1) / (2.0 * n_1)

w_0, w_1

In [ ]:
df['install'] = df['install'].astype('category')
df.info()

In [ ]:
CAT_NAMES = ['campaignId', 'platform', 'softwareVersion',
             'sourceGameId', 'connectionType', 'deviceType']
CONT_NAMES = ['startCount', 'viewCount', 'clickCount', 
              'installCount', 'startCount1d', 'startCount7d', 
              'timeSinceLastStart']

splits = RandomSplitter(valid_pct=0.2)(range_of(df))

dls = TabularDataLoaders.from_df(df, y_names='install', 
                                 cat_names=CAT_NAMES,
                                 cont_names=CONT_NAMES,
                                 procs=[Categorify, FillMissing, Normalize],
                                 splits=splits)

In [ ]:
dls.xs

In [ ]:
dls.valid_ds

In [ ]:
weights = [w_0, w_1]
class_weights=torch.FloatTensor(weights).cuda()
loss_func = CrossEntropyLossFlat(weights=class_weights)
roc_auc = RocAucBinary()

learn = tabular_learner(dls, loss_func=loss_func, metrics=roc_auc)
learn.loss_func

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_one_cycle(3)

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(3)